<a href="https://colab.research.google.com/github/beloveddie/AI-Craft/blob/main/Simple_LLM_Application_with_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation




In [2]:
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.1 MB/s eta 0:00:00


# LangSmith
LangSmith helps us to inspect what exactly is going on inside our chain or agent.


In [5]:
from google.colab import userdata
LANGCHAIN_TRACING_V2 = userdata.get("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY = userdata.get("LANGSMITH_API_KEY")

!export LANGCHAIN_TRACING_V2=LANGCHAIN_TRACING_V2
!export LANGCHAIN_API_KEY=LANGCHAIN_API_KEY

# Using Language Models

Let's practice how to use language models by itself. LangChain provides support for many different language models that we can use interchangably.

I'll be using Cohere's command R+ -preferentially.

In [8]:
!pip install -qU langchain-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [9]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r")

··········


Let's first use the model directly. `ChatModel`s are instances of LangChain "Runnables", which means that they expose a standard interface to interacting with them.

To just simply call the model we can pass a list of messages to the `.invoke` method.

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to Italian."),
    HumanMessage(content="Hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'd0c655d1-3166-4492-96a9-436912f7b99d', 'token_count': {'input_tokens': 79, 'output_tokens': 3}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'd0c655d1-3166-4492-96a9-436912f7b99d', 'token_count': {'input_tokens': 79, 'output_tokens': 3}}, id='run-166aafc9-e1c4-4be4-85cf-2d8180681ac0-0')

# OutputParsers

Notice that the response from the model is an `AIMessage` instance. This contains the response as a string alongside other metadata such as the `additional_kwargs`, `citations`, `search_results`, `search_queries`, etc.
Most times we just want to work with the string response. We can parse out just this string response using a simple output parser.

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

One way to use it is to use it by itself. For example, we could save the result of the language model call and then pass it to the parser.

In [13]:
result = model.invoke(messages)

In [14]:
parser.invoke(result)

'Ciao!'

More commonly, we can "chain" the model alongside with this output parser. The parser will get called each time the chain is invoked.

This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

In [15]:
chain = model | parser

In [16]:
chain.invoke(messages)

'Ciao!'

# Prompt Templates

PromptTemplates are a concept in LangChain designed to assist with this transformation (adding a system message or formatting a template with the user input).

They take in the raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a PromptTemplate here. It will take in two user variables:

* `language`: The language to translate the text into
* `text`: The actual text to be translated.


In [17]:
from langchain_core.prompts import ChatPromptTemplate

template = """Translate the following from English to {language}:
{text}"""

prompt = ChatPromptTemplate.from_template(template)

In [18]:
result = prompt.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[HumanMessage(content='Translate the following from English to italian:\nhi')])

In [19]:
system_template = "Translate the following into {language}:"

Next, we can create the PromptTemplate. This will be a combination of the `system_template` as well as a simpler template for where to put the text to be translated.

In [20]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", "{text}")
    ]
)

In [21]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [22]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

# Chaining together components with LCEL

We can easilt combine this with the model and the output parser from above using the pipe (`|`) operator:

In [23]:
chain = prompt_template | model | parser

chain.invoke({"language": "italian", "text": "hi"})

'Ciao!'

This is a simple example of using `LangChain Expression Language (LCEL)` to chain together LangChain modules. There are several benefits to this approach, including optimized streaming and tracing support.

# Serving with LangServe

Now that we've bult ann application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API.

In [24]:
!pip install -q langserve["all"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.9 MB/s eta 0:00:00


# Client

Now let's set up a client for programmatically interacting with our service. We can easily do this using the `langserve.RemoteRunnable`. Using this, we can interact with the served chain as if it were running client-side.

In [25]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

ConnectError: [Errno 99] Cannot assign requested address